# Implementazione del Perceptron - inizio parte 1

In [ ]:
import pandas as pd
import numpy as np
path = "C:\\Users\\ianto\\Desktop\\Corso Python per l analisi dei dati\\File\\"

In [ ]:
def preprocessing(path):
    iris = pd.read_csv(filepath_or_buffer=path + 'IrisDataset.csv',
            sep = ";", 
            header = None,
            names = ["rownumber","sepal_length","sepal_width","petal_length","petal_width","class"],
            skiprows = 4
            )
    
    iris_working = iris.copy()
    iris_working = iris_working[iris_working["class"].isin(["Iris setosa","Iris versicolor"])]
    iris_working = iris_working.sample(frac=1, 
                                       random_state = 0)
    iris_working = iris_working.drop(["rownumber"], 
                                     axis=1)
    
    
    numero_righe = len(iris_working)
    percentuale = 50/100
    soglia = numero_righe*percentuale
    lista_colonne = list(iris_working.columns)
    lista_colonne.remove("class")
    for colonna in lista_colonne:
        numero_null = len(iris_working[iris_working[colonna].isna()])
        if numero_null > soglia:
            iris_working.drop(colonna, axis=1)     
    
    iris_working.loc[iris_working["class"] == "Iris setosa", "class"] = 1
    iris_working.loc[iris_working["class"] == "Iris versicolor", "class"] = -1
    
    percentuale_training = 70/100
    numero_righe_training = int(numero_righe*percentuale_training)
    iris_training = iris_working.iloc[0:numero_righe_training].copy()
    iris_test = iris_working.iloc[numero_righe_training:].copy()
    
    for colonna in lista_colonne:
        media = iris_training[colonna].mean()
        iris_training[colonna] = iris_training[colonna].fillna(value = media)
        iris_test[colonna] = iris_test[colonna].fillna(value = media)
        
    for colonna in lista_colonne:
        media = iris_training[colonna].mean()
        deviazione_standard = iris_training[colonna].std()
        iris_training[colonna] = (iris_training[colonna] - media) / deviazione_standard
        iris_test[colonna] = (iris_test[colonna] - media) / deviazione_standard
            
    iris_training["weight"] = 1
    iris_test["weight"] = 1
    
    x1 = iris_training[["sepal_length","sepal_width","petal_length","petal_width","weight"]].values
    y1 = iris_training[["class"]].values

    x2 = iris_test[["sepal_length","sepal_width","petal_length","petal_width","weight"]].values
    y2 = iris_test[["class"]].values

    return [iris, iris_training, iris_test, x1, y1, x2, y2]

In [ ]:
 iris, iris_training, iris_test, x1, y1, x2, y2 = preprocessing(path)

In [ ]:
iris_training

creo un vettore w della stessa lunghezza di una riga di x1 con tutti 0

In [ ]:
x1

In [ ]:
x1.shape

In [ ]:
w = np.zeros(x1.shape[1])

In [ ]:
w

In [ ]:
x1[0]

In [ ]:
w[0] * x1[0][0] + w[1]*x1[0][1] + w[2]*x1[0][2] +  w[3]*x1[0][3] + w[4]*x1[0][4]

calcolo il prodotto scalare tra la prima riga di x1 e w

In [ ]:
x1_dots_w = np.dot(x1[0],w)  
x1_dots_w

se il prodotto scalare calcolato è maggiore o uguale di 0 allora il valore di class predetto sarà 1, altrimenti -1

In [ ]:
if x1_dots_w >= 0:
    predizione = 1
else:
    predizione = -1
    
predizione

raggruppiamo il codice precedente in una funzione

In [ ]:
def predici(x,w):
    x_dots_w = np.dot(x,w) 
    if x_dots_w >= 0:
        predizione = 1
    else:
        predizione = -1
    return predizione


testiamo la funzione con la prima riga di x e w

In [ ]:
predizione = predici(x1[0],w)

In [ ]:
predizione

In [ ]:
y1[0]

a questo punto possiamo aggiornare il vettore w tramite la formula

w = w + 0.1*(class[0] - predizione[0]) * x[0]

In [ ]:
w = w + 0.1 *(y1[0]-predizione)*x1[0]
w

In [ ]:
predizione = predici(x1[1],w)

In [ ]:
predizione

In [ ]:
y1[1]

In [ ]:
w = w + 0.1 *(y1[1]-predizione)*x1[1]
w

Iteriamo questo procedimento per tutte le righe di x1

In [ ]:
w = np.zeros(x1.shape[1])
eta = 0.1 
for i in range(x1.shape[0]):
    predizione = predici(x1[i],w)
    w = w + eta * (y1[i]-predizione) * x1[i]
    print(i)
w

# Implementazione del Perceptron - fine parte 1

# Implementazione del Perceptron - inizio parte 2

Nota didattica: prima di iniziare questa lezione, rilancia il codice della parte precedente se non lo hai già fatto

Ripetiamo questo procedimento un certo numero di volte(epoche).

In [ ]:
w = np.zeros(x1.shape[1])
epoche = 10
for i1 in range(epoche): 
    for i in range(x1.shape[0]):
        predizione = predici(x1[i],w)
        w = w + eta * (y1[i]-predizione) * x1[i]
w

Miglioriamo l'algoritmo in modo che;

- termini quando in un'epoca tutte le predizioni sono corrette

- riporti il numero di errori ad ogni epoca

In [ ]:
w = np.zeros(x1.shape[1])
eta = 0.1 
epoche = 10
iterazioni_eseguite = 0
lista_errori = []
for i1 in range(epoche): 
    iterazioni_eseguite = iterazioni_eseguite + 1
    errori = 0
    for i in range(x1.shape[0]):
        predizione = predici(x1[i],w)
        w = w + eta * (y1[i]-predizione) * x1[i]
        if (y1[i]-predizione) != 0:
            errori = errori + 1
    lista_errori.append(errori)
    if(errori == 0):
        break

In [ ]:
lista_errori

In [ ]:
w

Utilizziamo w per predirre la classe di x2 e contare il numero di errori

In [ ]:
predizioni = []
errori = 0
for i in range(x2.shape[0]):
    predizione = predici(x2[i],w)
    predizioni.append(predizione)
    if predizione != y2[i]:
        errori = errori + 1

accuratezza = (x2.shape[0] - errori) / x2.shape[0]
accuratezza, errori

Colleghiamo la colonna predizioni al dataframe iris_test

In [ ]:
iris_test["predizione"] = predizioni
iris_test

Decodifichiamo 1 e -1 in Iris setosa e Iris Versicolor

In [ ]:
iris_test.loc[iris_test["predizione"] == 1, "predizione"] =  "Iris setosa" 
iris_test.loc[iris_test["predizione"] == -1, "predizione"] = "Iris versicolor"

In [ ]:
iris_test

Recuperiamo i dati originali dal dataframe iris

In [ ]:
iris_previsioni = pd.concat(objs = [iris, iris_test[["predizione"]]], 
                            join = "inner",  
                            axis = 1)
iris_previsioni

Filtriamo gli errori

In [ ]:
iris_previsioni[iris_previsioni["class"] != iris_previsioni["predizione"]]